In [9]:
# 1. 필수 라이브러리 설치
!pip install -q langgraph langchain faiss-cpu python-dotenv groq sentence-transformers


In [1]:
from typing import TypedDict, List
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langgraph.graph import MessagesState
from langchain_core.tools import tool
import os

In [2]:
# 메시지 상태를 기반으로 하는 Agent 상태 정의
class AgentState(MessagesState):
    pass


In [3]:
# 임베딩 모델 설정
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 저장된 벡터 DB 로드
menu_db = FAISS.load_local(
    "C:/mylangchain/langchain_basic/data/menu_db",
    embeddings_model,
    allow_dangerous_deserialization=True
)


C:\Users\user\AppData\Local\Temp\ipykernel_11028\918181504.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
@tool
def search_cafe_menu(query: str) -> str:
    """카페 메뉴 DB에서 해당 메뉴 정보를 검색합니다."""
    docs = menu_db.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in docs])


In [5]:
import requests
from langchain_community.chat_models import ChatOllama

class QoobLLM:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.api_url = "https://api.qoob.ai/v1/chat/completions"
    
    def __call__(self, messages: list[dict]) -> dict:
        response = requests.post(
            self.api_url,
            headers={
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json",
            },
            json={
                "model": "meta-llama/llama-4-scout-17b-16e-instruct",
                "messages": messages,
                "temperature": 0.3,
            },
        )
        return response.json()

# Ollama LLM 인스턴스 생성 (qwen3:1.7b 모델 사용)
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_predict=1500
)

C:\Users\user\AppData\Local\Temp\ipykernel_11028\3715236935.py:25: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [6]:
from langchain_core.messages import AIMessage

def agent_node(state: AgentState):
    tool_call = {
        "id": "dummy-id",
        "name": "search_cafe_menu",
        "arguments": '{"query": "아메리카노"}'
    }
    return {
        "messages": state["messages"] + [
            AIMessage(content="에이전트 응답", tool_calls=[tool_call])
        ]
    }


In [7]:
from langchain_core.messages import BaseMessage, AIMessage, ToolMessage
from typing import List, TypedDict
import json
from langgraph.graph import StateGraph, END

# 1. 상태 정의 (필수)
class AgentState(TypedDict):
    messages: List[BaseMessage]
    # 필요한 경우 추가 필드:
    # tool_results: List[dict]

# 2. 노드 함수 정의 (필수)
def tool_node(state: AgentState):
    return {
        "messages": state["messages"] + [
            ToolMessage(content="도구 실행 결과", tool_call_id="dummy-id")
        ]
    }

def tool_node(state: AgentState):
    """도구 실행 노드"""
    return {"messages": state["messages"] + [ToolMessage(content="도구 실행 결과", tool_call_id="dummy-id")]}
    # 실제로는 tool_call_id는 마지막 AIMessage의 tool_call.id와 일치해야 하지만 여기선 placeholder 사용

# 3. 조건부 분기 함수 (필수)
def agent_node(state: AgentState):
    tool_call = {
        "id": "dummy-id",
        "function": {
            "name": "search_cafe_menu",
            "arguments": '{"query": "아메리카노"}'
        }
    }
    return {
        "messages": state["messages"] + [
            AIMessage(content="에이전트 응답", tool_calls=[tool_call])
        ]
    }
    
    # 1) tools_condition 함수 정의
def tools_condition(state: AgentState) -> str:
    last_message = state["messages"][-1]

    from langchain_core.messages import ToolMessage
    if isinstance(last_message, ToolMessage):
        return "end"
    return "tool" if hasattr(last_message, "tool_calls") else "end"

# 2) builder 생성 및 설정
builder = StateGraph(AgentState)
builder.add_node("agent", agent_node)
builder.add_node("tool", tool_node)
builder.add_edge("tool", "agent")
builder.add_conditional_edges("agent", tools_condition, {"tool": "tool", "end": END})
builder.set_entry_point("agent")

# 3) graph 컴파일
graph = builder.compile()


# 4. 그래프 구성 (수정 버전)
builder = StateGraph(AgentState)

# 노드 등록
builder.add_node("agent", agent_node)
builder.add_node("tool", tool_node)

# 엣지 연결
builder.add_edge("tool", "agent")  # 도구 실행 후 에이전트로 복귀

# 조건부 분기 설정
builder.add_conditional_edges(
    "agent",
    tools_condition,
    {"tool": "tool", "end": END}  # 조건에 따른 다음 노드 매핑
)

# 진입점 설정 (필수)
builder.set_entry_point("agent")

# 그래프 컴파일
graph = builder.compile()


In [8]:
from langchain_core.messages import HumanMessage

test_queries = [
    "아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.",
    "라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?",
    "디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요."
]

for query in test_queries:
    print("==== 사용자 질문 ====")
    print(query)
    print("==== 응답 결과 ====")

    inputs = {"messages": [HumanMessage(content=query)]}
    result = graph.invoke(inputs)

    for msg in result["messages"]:
        role = type(msg).__name__
        print(f"{role}: {msg.content}\n")

    print("=" * 50 + "\n")


==== 사용자 질문 ====
아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.
==== 응답 결과 ====


TypeError: tool_call() got an unexpected keyword argument 'function'